# How to connect Jupyter notebook (local instance) with Amazon S3

## AWS Credentials 

Create your access key:
- go to aws console
- IAM -> Users -> <User> -> Security credentials -> Download file (Store the keys in a secure location)

Install the AWS Command Line Interface on macOS:

$ pip install awscli --upgrade --user


$ aws configure

AWS Access Key ID [None]: [Access-key-from-csv-file]

AWS Secret Access Key [None]: [Secret-Access-key-from-csv-file]

Default region name [None]: [Enter]

Default output format [None]: [Enter]

Your credentials should be saved in ~/.aws/credentials file

## Connect to S3 in Jupyter Notebook  

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell"

In [2]:
import configparser
import os
config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get('default', "aws_access_key_id") 
access_key = config.get('default', "aws_secret_access_key")

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", access_id)
hadoop_conf.set("fs.s3n.awsSecretAccessKey", access_key)

In [4]:
df=spark.read.orc("s3n://miquido-ds-test-backet/events_processing/events_processing_df_002_orc")
df.limit(5).toPandas()[['meta_id', 'meta_timestamp', 'data_device_device_type']].head()

,meta_id,meta_timestamp,data_device_device_type
0,333A,100000000,mobile
1,333A,100000020,None
2,333A,100000040,None
3,333A,100000060,mobile
4,333A,100000080,None
